In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Topic Modelling: Going Beyond Tokens Outputs

In [28]:
!pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=7c789cf3ff40ac642b56266ce9ae3622a1231eef4b1b721bdafe47076b3d247f
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [29]:
import pandas as pd
import numpy as np
import re
import nltk
import collections
import plotly.graph_objects as go

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import ngrams

from rake_nltk import Rake

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [16]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [91]:
# Load data
df = pd.read_csv('/content/drive/My Drive/colabdata/news  input/news_input.csv')
df.head()

,text,company,label
0,Asia-Pacific Hydrocolloids Market Overview 201...,"INDOKING ANEKA AGAR AGAR INDUSTRI,PT",neutral
1,"China Cosmetics Market Report, 2019-2025 Featu...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral
2,[PDF Report] Global Facial Cleanser Market 202...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral
3,Global Facial Cleanser Market Strategies Emplo...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral
4,"Global Facial Cleanser Market Growth Dynamics,...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral


In [92]:

df['cleaned_content'] = df['text'].apply(lambda x: re.sub('\s+', ' ', re.sub('[^0-9a-z\s-]',' ', x.lower())).strip())

df.head()

,text,company,label,cleaned_content
0,Asia-Pacific Hydrocolloids Market Overview 201...,"INDOKING ANEKA AGAR AGAR INDUSTRI,PT",neutral,asia-pacific hydrocolloids market overview 201...
1,"China Cosmetics Market Report, 2019-2025 Featu...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,china cosmetics market report 2019-2025 featur...
2,[PDF Report] Global Facial Cleanser Market 202...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,pdf report global facial cleanser market 2020-...
3,Global Facial Cleanser Market Strategies Emplo...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market strategies emplo...
4,"Global Facial Cleanser Market Growth Dynamics,...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market growth dynamics ...


In [93]:
df['token_list'] = df.apply(lambda row: nltk.word_tokenize(row['cleaned_content']), axis=1)
df.head()

,text,company,label,cleaned_content,token_list
0,Asia-Pacific Hydrocolloids Market Overview 201...,"INDOKING ANEKA AGAR AGAR INDUSTRI,PT",neutral,asia-pacific hydrocolloids market overview 201...,"[asia-pacific, hydrocolloids, market, overview..."
1,"China Cosmetics Market Report, 2019-2025 Featu...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,china cosmetics market report 2019-2025 featur...,"[china, cosmetics, market, report, 2019-2025, ..."
2,[PDF Report] Global Facial Cleanser Market 202...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,pdf report global facial cleanser market 2020-...,"[pdf, report, global, facial, cleanser, market..."
3,Global Facial Cleanser Market Strategies Emplo...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market strategies emplo...,"[global, facial, cleanser, market, strategies,..."
4,"Global Facial Cleanser Market Growth Dynamics,...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market growth dynamics ...,"[global, facial, cleanser, market, growth, dyn..."


In [94]:
# initialize stopwords from nltk
stop_words = stopwords.words('english')

# with additional missing terms
stop_words.extend(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l','m','n','o','p','q','r','s','t', 'u', 'v', 'w', 'x', 'y', 'z', "about", "across", "after", "all", "also", "an", "and", "another", "added",
"any", "are", "as", "at", "basically", "be", "because", 'become', "been", "before", "being", "between","both", "but", "by","came","can","come","could","did","do","does","each","else","every","either","especially", "for","from","get","given","gets",
'give','gives',"got","goes","had","has","have","he","her","here","him","himself","his","how","if","in","into","is","it","its","just","lands","like","make","making", "made", "many","may","me","might","more","most","much","must","my","never","provide", 
"provides", "perhaps","no","now","of","on","only","or","other", "our","out","over","re","said","same","see","should","since","so","some","still","such","seeing", "see", "take","than","that","the","their","them","then","there",
"these","they","this","those","through","to","too","under","up","use","using","used", "underway", "very","want","was","way","we","well","were","what","when","where","which","while","whilst","who","will","with","would","you","your", 
'etc', 'via', 'eg', 'pdf', 'company', 'corporation', 'ltd', 'limited', 'private', 'public']) 

# remove stopwords

df['token_list'] = df['token_list'].apply(lambda x: [item for item in x if item not in stop_words])

df.head()


,text,company,label,cleaned_content,token_list
0,Asia-Pacific Hydrocolloids Market Overview 201...,"INDOKING ANEKA AGAR AGAR INDUSTRI,PT",neutral,asia-pacific hydrocolloids market overview 201...,"[asia-pacific, hydrocolloids, market, overview..."
1,"China Cosmetics Market Report, 2019-2025 Featu...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,china cosmetics market report 2019-2025 featur...,"[china, cosmetics, market, report, 2019-2025, ..."
2,[PDF Report] Global Facial Cleanser Market 202...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,pdf report global facial cleanser market 2020-...,"[report, global, facial, cleanser, market, 202..."
3,Global Facial Cleanser Market Strategies Emplo...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market strategies emplo...,"[global, facial, cleanser, market, strategies,..."
4,"Global Facial Cleanser Market Growth Dynamics,...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market growth dynamics ...,"[global, facial, cleanser, market, growth, dyn..."


In [95]:
# initiate nltk lemmatiser

wordnet_lemmatizer = WordNetLemmatizer()

# lemmatise words

df['token_list'] = df['token_list'].apply(lambda x: [wordnet_lemmatizer.lemmatize(y) for y in x]) 


df.head()

,text,company,label,cleaned_content,token_list
0,Asia-Pacific Hydrocolloids Market Overview 201...,"INDOKING ANEKA AGAR AGAR INDUSTRI,PT",neutral,asia-pacific hydrocolloids market overview 201...,"[asia-pacific, hydrocolloid, market, overview,..."
1,"China Cosmetics Market Report, 2019-2025 Featu...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,china cosmetics market report 2019-2025 featur...,"[china, cosmetic, market, report, 2019-2025, f..."
2,[PDF Report] Global Facial Cleanser Market 202...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,pdf report global facial cleanser market 2020-...,"[report, global, facial, cleanser, market, 202..."
3,Global Facial Cleanser Market Strategies Emplo...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market strategies emplo...,"[global, facial, cleanser, market, strategy, e..."
4,"Global Facial Cleanser Market Growth Dynamics,...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market growth dynamics ...,"[global, facial, cleanser, market, growth, dyn..."


In [96]:
# initialise the count vectorizer

vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (1, 2))
                            
# join the processed data to be vectorised

vectors = []

for index, row in df.iterrows():
    vectors.append(", ".join(row['token_list']))

vectorised = vectorizer.fit_transform(vectors)

print(vectorised)

  (0, 2227)	1
  (0, 13925)	1
  (0, 9499)	1
  (0, 12143)	1
  (0, 13856)	1
  (0, 524)	1
  (0, 2242)	1
  (0, 13933)	1
  (0, 9501)	1
  (0, 12189)	1
  (0, 13857)	1
  (1, 12143)	1
  (1, 524)	1
  (1, 4265)	1
  (1, 5333)	1
  (1, 16076)	1
  (1, 578)	1
  (1, 7512)	1
  (1, 4281)	1
  (1, 5336)	1
  (1, 12196)	1
  (1, 16078)	1
  (1, 527)	1
  (1, 581)	1
  (2, 12143)	1
  :	:
  (1321, 18387)	1
  (1321, 10427)	1
  (1321, 7553)	1
  (1321, 18388)	1
  (1321, 10351)	1
  (1322, 4343)	1
  (1322, 7768)	1
  (1322, 2811)	1
  (1322, 18722)	1
  (1322, 17859)	1
  (1322, 7776)	1
  (1322, 20127)	2
  (1322, 18481)	1
  (1322, 10425)	1
  (1322, 2485)	1
  (1322, 4666)	1
  (1322, 10426)	1
  (1322, 2812)	1
  (1322, 4381)	1
  (1322, 20130)	1
  (1322, 18484)	1
  (1322, 17867)	1
  (1322, 18723)	1
  (1322, 2486)	1
  (1322, 20128)	1


In [97]:
# initisalise LDA Model

lda_model = LatentDirichletAllocation(n_components = 10, # number of topics
                                  random_state = 10,          # random state
                                  evaluate_every = -1,      # compute perplexity every n iters, default: Don't
                                  n_jobs = -1,              # Use all available CPUs
                                 )

lda_output = lda_model.fit_transform(vectorised)

# column names

topic_names = ["Topic" + str(i) for i in range(1, lda_model.n_components + 1)]

# make the pandas dataframe

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns = topic_names)

# get dominant topic for each document

dominant_topic = (np.argmax(df_document_topic.values, axis=1)+1)
df_document_topic['Dominant_topic'] = dominant_topic

# join to original dataframes

df = pd.merge(df, df_document_topic, left_index=True, right_index=True, how='outer')
df.head()

,text,company,label,cleaned_content,token_list,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Dominant_topic
0,Asia-Pacific Hydrocolloids Market Overview 201...,"INDOKING ANEKA AGAR AGAR INDUSTRI,PT",neutral,asia-pacific hydrocolloids market overview 201...,"[asia-pacific, hydrocolloid, market, overview,...",0.01,0.01,0.01,0.92,0.01,0.01,0.01,0.01,0.01,0.01,4
1,"China Cosmetics Market Report, 2019-2025 Featu...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,china cosmetics market report 2019-2025 featur...,"[china, cosmetic, market, report, 2019-2025, f...",0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.94,10
2,[PDF Report] Global Facial Cleanser Market 202...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,pdf report global facial cleanser market 2020-...,"[report, global, facial, cleanser, market, 202...",0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.94,0.01,0.01,8
3,Global Facial Cleanser Market Strategies Emplo...,"SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market strategies emplo...,"[global, facial, cleanser, market, strategy, e...",0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.94,10
4,"Global Facial Cleanser Market Growth Dynamics,...","SHANGHAI INOHERB COSMETICS CO., LTD.",neutral,global facial cleanser market growth dynamics ...,"[global, facial, cleanser, market, growth, dyn...",0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.94,0.01,0.01,8


In [98]:
keywords = np.array(vectorizer.get_feature_names())

topic_keywords = []

for topic_weights in lda_model.components_:
    top_keyword_locs = (-topic_weights).argsort()[:20]
    topic_keywords.append(keywords.take(top_keyword_locs))

for i in topic_keywords:
    print(i)

['group' 'china' 'bhd' 'greenland' 'property' 'sale' 'year' 'resource'
 'gas' 'kuala' 'lumpur' 'kuala lumpur' 'resource gas' 'china resource'
 'rating' 'gas group' 'holding' '1193' 'fitch' 'million']
['keppel' 'group' 'first' 'sponsor' 'first sponsor' 'million' 'bhd'
 'ann joo' 'ann' 'joo' 'china' 'bribe' 'sponsor group' 'singapore' 'sgx'
 'steel' 'co' 'contract' 'brazil' 'adn']
['hiap' 'hoe' 'hiap hoe' 'holding' 'digital' 'group' 'skyworth'
 'skyworth digital' 'digital holding' 'property' 'hkg' 'holding hkg'
 'year' 'million' 'read' 'stock' 'singapore' 'share' 'hkg 751' '751']
['bak kwa' 'kwa' 'bak' 'group' 'guan' 'price' 'market' '2020' 'chinese'
 'kim' 'joo guan' 'kim joo' 'joo' 'first' 'cny' 'ratio' 'holding'
 'first sponsor' 'sponsor' 'co']
['bhd' 'group' 'holding' 'fantasia' 'fantasia holding' 'holding group'
 'international' 'co' 'mulpha' 'group co' 'mulpha international' 'hkg'
 '1777' 'hkg 1777' 'year' 'co hkg' 'international bhd' 'guocoland'
 'million' 'kuala']
['new' 'busines

In [99]:
# Remove duplicates from the keywords extracted from the topic modelling output. This is because we want to limit the amount of Ideas that go across multiple topics. We want to stick to 1 idea to 1 topic
dupes = []

for i in topic_keywords:
    for j in i:
        dupes.append(str(j))

dupes = [item for item, count in collections.Counter(dupes).items() if count > 1]

topic_keywords_processed = []

for i in topic_keywords:
    tmp = []
    for j in i:
        if str(j) not in dupes:
            tmp.append(str(j))
        else:
            pass

    topic_keywords_processed.append(tmp)
    
for i in topic_keywords_processed:
    print(i)

['greenland', 'sale', 'resource gas', 'china resource', 'rating', 'gas group', '1193', 'fitch']
['keppel', 'ann joo', 'ann', 'bribe', 'sponsor group', 'sgx', 'steel', 'contract', 'brazil', 'adn']
['hiap', 'hoe', 'hiap hoe', 'digital', 'skyworth', 'skyworth digital', 'digital holding', 'read', 'share', 'hkg 751', '751']
['bak kwa', 'kwa', 'bak', 'guan', 'price', '2020', 'kim', 'joo guan', 'kim joo', 'cny', 'ratio']
['fantasia', 'fantasia holding', 'holding group', 'mulpha', 'group co', 'mulpha international', '1777', 'hkg 1777', 'co hkg', 'international bhd']
['business', 'exxon', 'york', 'new york', 'lawsuit', 'international holding', 'opportunity', 'cheil', 'investment', 'oil']
['panoil', 'indonesia', 'aipl', 'bank', 'loss', 'say', 'management', 'quarter', 'xinwei', 'sanko', 'panoil petroleum', 'company']
['lng', 'energy', 'sabah', 'sabah energy', 'singapore lng', 'terminal', 'slng', 'global', 'dividend', 'matheson']
['sen', 'sec', 'woodbridge', 'shapiro', 'cent']
['three', 'resource 

In [100]:
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords_processed)
df_topic_keywords.columns = ['Term '+ str(i) for i in range(1, df_topic_keywords.shape[1] + 1)]
df_topic_keywords['Topic_keywords'] = df_topic_keywords.values.tolist()
df_topic_keywords['Topic_number'] = df_topic_keywords.index + 1
df_topic_keywords = df_topic_keywords[['Topic_keywords', 'Topic_number']]

# Remove None from lists
tmp = []

for i in df_topic_keywords['Topic_keywords']:
    tmp.append([x for x in i if x is not None])

df_topic_keywords['Topic_keywords'] = tmp

# Merge key terms back to main frame
df = pd.merge(df, df_topic_keywords, left_on='Dominant_topic', right_on='Topic_number')
del df['Topic_number']

df.head()

,text,company,label,cleaned_content,token_list,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Dominant_topic,Topic_keywords
0,Asia-Pacific Hydrocolloids Market Overview 201...,"INDOKING ANEKA AGAR AGAR INDUSTRI,PT",neutral,asia-pacific hydrocolloids market overview 201...,"[asia-pacific, hydrocolloid, market, overview,...",0.01,0.01,0.01,0.92,0.01,0.01,0.01,0.01,0.01,0.01,4,"[bak kwa, kwa, bak, guan, price, 2020, kim, jo..."
1,Bak Kwa prices in Singapore (CNY 2020) - what ...,KIM JOO GUAN PTE LTD,neutral,bak kwa prices in singapore cny 2020 - what is...,"[bak, kwa, price, singapore, cny, 2020, -, worth]",0.01,0.01,0.01,0.94,0.01,0.01,0.01,0.01,0.01,0.01,4,"[bak kwa, kwa, bak, guan, price, 2020, kim, jo..."
2,Bak Kwas worth queueing up for this CNY (2020),KIM JOO GUAN PTE LTD,positive,bak kwas worth queueing up for this cny 2020,"[bak, kwa, worth, queueing, cny, 2020]",0.01,0.01,0.01,0.92,0.01,0.01,0.01,0.01,0.01,0.01,4,"[bak kwa, kwa, bak, guan, price, 2020, kim, jo..."
3,Bak kwa sellers blame price rises on CNY deman...,KIM JOO GUAN PTE LTD,neutral,bak kwa sellers blame price rises on cny deman...,"[bak, kwa, seller, blame, price, rise, cny, de...",0.01,0.01,0.01,0.95,0.01,0.01,0.01,0.01,0.01,0.01,4,"[bak kwa, kwa, bak, guan, price, 2020, kim, jo..."
4,Bak Kwa prices in Singapore 2019 - Bulk orders...,KIM JOO GUAN PTE LTD,neutral,bak kwa prices in singapore 2019 - bulk orders...,"[bak, kwa, price, singapore, 2019, -, bulk, or...",0.01,0.43,0.01,0.52,0.01,0.01,0.01,0.01,0.01,0.01,4,"[bak kwa, kwa, bak, guan, price, 2020, kim, jo..."


In [101]:
all_results = []

for i in df['Dominant_topic'].unique():
    topic = df[df['Dominant_topic'] == i]
    topic = topic.copy()

    key_words = []
    
    # run keyword extraction
    for j in topic['cleaned_content'].values.tolist():
        r = Rake()
        a = r.extract_keywords_from_text(j)
        c = r.get_ranked_phrases_with_scores()

        for k in c:
            if k not in key_words:
                key_words.append(k)

    key_words = pd.DataFrame(key_words, columns =['score', 'term'])  
    key_words = key_words.sort_values('score', ascending=False)
    key_words = key_words.drop_duplicates(subset=['term'])
    key_words['topic_number'] = i
    
    key_words['term_list'] =  key_words.term.apply(lambda x: x.split())
    
    # find bigrams from key words to match against topic modelling output
    tmp_keywords = []

    for j in key_words.values.tolist():
        tmp = []

        bi_grams = ngrams(j[3], 2)
        
        for g in bi_grams:
            tmp.append(' '.join(g))
        
        for k in j[3]:
            tmp.append(k)
            # lemmatise words to match the lemmatised output of the topic modelling word extraction
            tmp.append(wordnet_lemmatizer.lemmatize(k))
            
        j.remove(j[3])
        j.append(list(set(tmp)))

        tmp_keywords.append(j)
    
    # mask key words against topic modelling output
    key_words = pd.DataFrame(tmp_keywords, columns =['score', 'term', 'topic_number', 'term_list'])  
    
    topic_keywords = topic['Topic_keywords'].values.tolist()
    topic_keywords = [item for sublist in topic_keywords for item in sublist]
    topic_keywords = list(set(topic_keywords))
    
    tmp = []

    for t in topic_keywords:

        mask = key_words.term_list.apply(lambda x: t in x)
        key_words_processed = key_words[mask]

        if key_words_processed.empty:
            pass
        else:
            for j in key_words_processed[['score', 'term', 'topic_number']].values.tolist():
                if j not in tmp:
                    tmp.append(j)

    key_words = pd.DataFrame(tmp, columns =['score', 'term', 'topic_number'])  

    # select the max score as the topic title
    top_key_words = key_words[key_words.score == key_words['score'].max()]

    # select the remaining keywords as child terms
    remaining_keywords = key_words[key_words.score != key_words['score'].max()]
    
    # if there are more than 1 keyword in the topic title, aggregate them with a / as a separatore
    top_key_words = top_key_words.copy()
    top_key_words = top_key_words.groupby(['score', 'topic_number']).agg({'term' : lambda x: ' / '.join(map(str, x))})
    top_key_words = top_key_words.reset_index()
    top_key_words['parent'] = ''

    # add 0.1 to the child keywords to identify then in the merged dataframe
    remaining_keywords = remaining_keywords.copy()
    remaining_keywords['topic_number'] = remaining_keywords['topic_number'] + 0.1
    remaining_keywords['parent'] = top_key_words['term'].values.tolist()[0]
    
    all_topics = pd.concat([top_key_words, remaining_keywords], sort=False)
    
    for t in all_topics.to_dict(orient='records'):
        all_results.append(t)

all_results.append({'score': 1000, 'topic_number' : 0.0, 'term' : '', 'parent' : ''})

all_topics_df = pd.DataFrame(all_results)
all_topics_df = all_topics_df.sort_values('topic_number', ascending=True)
all_topics_df = all_topics_df.loc[all_topics_df['score'] > 10.0]

display(all_topics_df)

,score,topic_number,term,parent
637,1000.000000,0.0,,
182,1866.547619,1.0,frank rating information disclosure form https...,
256,52.688013,1.1,greenland hong kong holdings limited,frank rating information disclosure form https...
251,10.714286,1.1,rating case,frank rating information disclosure form https...
250,11.214286,1.1,standalone rating,frank rating information disclosure form https...
...,...,...,...,...
42,25.000000,10.1,1 11 billion term loan,li zhongchu beijing shiji information technolo...
45,81.000000,10.1,first three bell 505 jet ranger x helicopters ...,li zhongchu beijing shiji information technolo...
47,16.000000,10.1,promoting three prescription drugs,li zhongchu beijing shiji information technolo...
41,25.000000,10.1,wirecard loses 8 3 billion,li zhongchu beijing shiji information technolo...


In [102]:
fig = go.Figure(go.Sunburst(
    labels = all_topics_df['term'].values.tolist(),
    parents = all_topics_df['parent'].values.tolist(),
    values = all_topics_df['score'].values.tolist()
))

fig.update_layout(margin = dict(t = 0, l = 0,  r = 0, b = 0))
fig.write_html("file.html")
fig.show()